## Bolukbasi et al. (2016) Quantifying and Reducing Gender Stereotypes in Word Embeddings

In [5]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions

## Part 1: Gender Bias in Word Embedding


### Step 1: Load data
We first load the word embedding trained on a corpus of Gutenberg texts consisting of 8246 English words and terms. The embedding maps each word into a 300-dimension vector. 

In [9]:
# load google news word2vec
#E = WordEmbedding('./embeddings/w2v_gnews_small.txt')
E = WordEmbedding('../embeddings/gutenberg_embeddings.txt')

# load professions
professions = load_professions()
profession_words = [p[0] for p in professions]

*** Reading data from ../embeddings/gutenberg_embeddings.txt
Got weird line  0.23161213 0.3326216 0.12401494 0.008499765 0.19889107 0.040334325 0.010168189 0.15632121 -0.06454332 -0.13223206 0.29689196 -0.11956101 0.12087899 -0.016669286 -0.0010117352 -0.0032466403 0.35093147 0.21012245 0.17422993 0.08760714 -0.24807881 -0.045603327 -0.09061381 -0.051755156 0.32621 0.08243399 -0.05821633 0.08455631 -0.19764413 -0.08475026 0.09770049 -0.04057578 -0.23146954 -0.020349184 0.046424106 0.08500739 0.062360015 -0.10232382 0.112622134 0.22026184 0.0762657 -0.084087804 -0.03204645 0.02017981 -0.091295 0.03855523 0.12979372 0.09027992 -0.06410779 0.20787267 0.051131696 0.07870032 -0.22296295 0.094810165 -0.20940404 -0.0032938076 0.17629741 0.12214556 -0.018406508 0.25541154 0.12578866 0.021828886 -0.057006184 -0.090764016 -0.41408795 0.18489662 0.069210954 -0.037505906 0.15761656 0.13942134 -0.13241915 0.25543803 -0.036114257 -0.1048695 0.02420018 -0.012577242 0.024264352 -0.04473054 0.057568956

### Step 2: Define gender direction

We define gender direction by the direction of she - he because they are frequent and do not have fewer alternative word senses (e.g., man can also refer to mankind). An alternative approach could be using PCA to define gender direction.

In [10]:
# gender direction
v_gender = E.diff('she', 'he')

### Step 3: Generating analogies of "Man: x :: Woman : y"

We show that the word embedding model generates gender-streotypical analogy pairs. 
To generate the analogy pairs, we use the analogy score that finds word pairs that are well aligned with gender direction as well as within a short distance from each other to preserve topic consistency.

In [11]:
# analogies gender
a_gender = E.best_analogies_dist_thresh(v_gender)

for (a,b,c) in a_gender:
    print(a+"-"+b)

Computing neighbors
Mean: 4.805639835802249
Median: 0.0
she-he
charmian-catesby
elizabeth-personae
maria-aguecheek
everidge-perdosa
katherine-florizel
katharine-malcolm
emilia-thersites
katherina-flavius
esther-haman
me-him
tamora-inus
cleopatra-enobarbus
forsake-prosper
jacinta-fabian
constance-bedford
bertram-senior
rachel-abimelech
fourteen-twentyfive
beef-pie
hortensio-biondello
reuben-joab
schooner-thrackles
bel-alcan
hair-beard
pacific-kansas
pink-patch
husband-wife
isabella-escalus
evadne-louis
jimmie-anyway
breadth-measured
bianca-grumio
olivia-es
lily-icy
samuel-joshua
rosaline-edm
cloak-shirt
fourteenth-brooklyn
hermione-camillo
fir-thicket
eros-gower
tossed-leaped
francisco-san
polixenes-agamemnon
lucetta-alcibiades
ladyship-lordship
climbed-marched
crimson-breadcrumbs
meadow-glacier
madame-marshal
l-stahl
boyet-menenius
townsend-aylmer
zadok-judas
ammon-chaldeans
bullocks-rams
leontes-fluellen
gloucester-exeter
reception-embassy
gentlewoman-lafeu
philadelphia-pittsburgh
mar

### Step 4: Analyzing gender bias in word vectors asscoiated with professions

Next, we show that many occupations are unintendedly associated with either male of female by projecting their word vectors onto the gender dimension. 

The script will output the profession words sorted with respect to the projection score in the direction of gender.

In [12]:
# profession analysis gender
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp[0:20], sp[-20:]

KeyError: 'accountant'

## Debiased word embedding

You can use debiaswe debias function to do the debiasing with word sets of your choosing

You can leave equalize_pairs and gender_specific_words blank when coming up with your own groups.

In [14]:
from debiaswe.debias import debias

ModuleNotFoundError: No module named 'we'

In [15]:
# Lets load some gender related word lists to help us with debiasing
with open('./data/definitional_pairs.json', "r") as f:
    defs = json.load(f)
print("definitional", defs)

with open('./data/equalize_pairs.json', "r") as f:
    equalize_pairs = json.load(f)

with open('./data/gender_specific_seed.json', "r") as f:
    gender_specific_words = json.load(f)
print("gender specific", len(gender_specific_words), gender_specific_words[:10])

definitional [['woman', 'man'], ['girl', 'boy'], ['she', 'he'], ['mother', 'father'], ['daughter', 'son'], ['gal', 'guy'], ['female', 'male'], ['her', 'his'], ['herself', 'himself'], ['Mary', 'John']]
gender specific 218 ['actress', 'actresses', 'aunt', 'aunts', 'bachelor', 'ballerina', 'barbershop', 'baritone', 'beard', 'beards']


In [8]:
debias(E, gender_specific_words, defs, equalize_pairs)

26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
{('CHAIRMAN', 'CHAIRWOMAN'), ('CONGRESSMAN', 'CONGRESSWOMAN'), ('SCHOOLBOY', 'SCHOOLGIRL'), ('Dads', 'Moms'), ('MALE', 'FEMALE'), ('himself', 'herself'), ('man', 'woman'), ('Man', 'Woman'), ('COUNCILMAN', 'COUNCILWOMAN'), ('Fella', 'Granny'), ('Boy', 'Girl'), ('congressman', 'congresswoman'), ('Dad', 'Mom'), ('COLT', 'FILLY'), ('HIS', 'HER'), ('Congressman', 'Congresswoman'), ('Nephew', 'Niece'), ('SPOKESMAN', 'SPOKESWOMAN'), ('brother', 'sister'), ('FATHER', 'MOTHER'), ('councilman', 'councilwoman'), ('men', 'women'), ('GRANDSONS', 'GRANDDAUGHTERS'), ('Men', 'Women'), ('FATHERS', 'MOTHERS'), ('His', 'Her'), ('GRANDFATHER', 'GRANDMOTHER'), ('Spokesman', 'Spokeswoman'), ('CATHOLIC_PRIEST', 'NUN'), ('catholic_priest', 'nun'), ('nephew', 'niece'), ('Sons', 'Daughters'), ('WIVES', 'HUSBANDS'), ('GRANDSON', 'GRANDDAUGHTER'), ('Fraternity', 'Sorority'), ('prostate_cancer', 'ovarian_cancer'), ('boy', 'girl'), ('

In [16]:
# profession analysis gender
sp_debiased = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp_debiased[0:20], sp_debiased[-20:]

KeyError: 'accountant'

In [10]:
# analogies gender
a_gender_debiased = E.best_analogies_dist_thresh(v_gender)

for (a,b,c) in a_gender_debiased:
    print(a+"-"+b)

Computing neighbors
Mean: 10.218597434053665
Median: 7.0
princess-prince
spokeswoman-spokesman
women-men
girls-boys
grandmother-grandfather
granddaughters-grandsons
mare-gelding
estrogen-testosterone
queens-kings
daughters-sons
females-males
mother-father
woman-man
gals-dudes
chairwoman-chairman
mom-dad
granddaughter-grandson
moms-dads
girl-boy
niece-nephew
motherhood-fatherhood
congresswoman-congressman
sisters-brothers
queen-king
mothers-fathers
she-he
schoolgirl-schoolboy
filly-colt
businesswoman-businessman
herself-himself
ex_boyfriend-ex_girlfriend
sorority-fraternity
sister-brother
aunt-uncle
ladies-gentlemen
husbands-wives
grandma-grandpa
councilwoman-councilman
daughter-son
convent-monastery
twin_sister-twin_brother
her-his
ovarian_cancer-prostate_cancer
female-male
actress-actor
lesbian-gay
compatriot-countryman
husband-younger_brother
gal-dude
hers-theirs
heroine-protagonist
feminism-feminist
actresses-actors
childhood-boyhood
waitress-waiter
kid-guy
me-him
mommy-daddy
aunts-